This notebook is a prototype of the rTMS Bayesian Optimization algorithm and will reflect the final step of pipeline after tetrahedral projection and FEM surface construction/registration

In [8]:
import os
import numpy as np
from fieldopt import geolib

In [3]:
#MOE package loading
from moe.optimal_learning.python.cpp_wrappers.domain import TensorProductDomain as cTensorProductDomain
from moe.optimal_learning.python.python_version.domain import TensorProductDomain
from moe.optimal_learning.python.geometry_utils import ClosedInterval
from moe.optimal_learning.python.cpp_wrappers.expected_improvement import ExpectedImprovement
from moe.optimal_learning.python.cpp_wrappers.expected_improvement import multistart_expected_improvement_optimization as meio
from moe.optimal_learning.python.data_containers import HistoricalData, SamplePoint
from moe.optimal_learning.python.cpp_wrappers.log_likelihood_mcmc import GaussianProcessLogLikelihoodMCMC
from moe.optimal_learning.python.default_priors import DefaultPrior
from moe.optimal_learning.python.python_version.optimization import GradientDescentOptimizer, GradientDescentParameters
from moe.optimal_learning.python.cpp_wrappers.optimization import GradientDescentOptimizer as cGDOpt
from moe.optimal_learning.python.cpp_wrappers.optimization import GradientDescentParameters as cGDParams

In [16]:
#Step 1: Load in files containing relevant information about search domain
proj_dir =  '/projects/jjeyachandra/rtms_optimize'
C_file = os.path.join(proj_dir,'output','quadratic_vec')
iR_file = os.path.join(proj_dir,'output','inverse_rot')
b_file = os.path.join(proj_dir,'output','param_bounds')
C = np.fromfile(C_file)
iR = np.fromfile(iR_file).reshape(3,3)
b = np.fromfile(b_file).reshape(3,2)

In [18]:
#Step 2: Specify the search domain
search_domain = TensorProductDomain([
    ClosedInterval(b[0,0],b[0,1]), #X coord on quadratic surface
    ClosedInterval(b[1,0],b[1,1]), #Y coord on quadratic surface
    ClosedInterval(0,180) #Rotational angle
])

c_search_domain = cTensorProductDomain([
    ClosedInterval(b[0,0],b[0,1]), 
    ClosedInterval(b[1,0],b[1,1]),
    ClosedInterval(0,180)
])

In [ ]:
#Step 3: Specify historical data 